### Building Autonomous Agents to Create Analysis Reports

In [1]:
# We scrape several Artificial Intelligence news

import requests
from newspaper import Article # https://github.com/codelucas/newspaper
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = [
    "https://www.artificialintelligence-news.com/2023/05/23/meta-open-source-speech-ai-models-support-over-1100-languages/",
    "https://www.artificialintelligence-news.com/2023/05/18/beijing-launches-campaign-against-ai-generated-misinformation/"
    "https://www.artificialintelligence-news.com/2023/05/16/openai-ceo-ai-regulation-is-essential/",
    "https://www.artificialintelligence-news.com/2023/05/15/jay-migliaccio-ibm-watson-on-leveraging-ai-to-improve-productivity/",
    "https://www.artificialintelligence-news.com/2023/05/15/iurii-milovanov-softserve-how-ai-ml-is-helping-boost-innovation-and-personalisation/",
    "https://www.artificialintelligence-news.com/2023/05/11/ai-and-big-data-expo-north-america-begins-in-less-than-one-week/",
    "https://www.artificialintelligence-news.com/2023/05/11/eu-committees-green-light-ai-act/",
    "https://www.artificialintelligence-news.com/2023/05/09/wozniak-warns-ai-will-power-next-gen-scams/",
    "https://www.artificialintelligence-news.com/2023/05/09/infocepts-ceo-shashank-garg-on-the-da-market-shifts-and-impact-of-ai-on-data-analytics/",
    "https://www.artificialintelligence-news.com/2023/05/02/ai-godfather-warns-dangers-and-quits-google/",
    "https://www.artificialintelligence-news.com/2023/04/28/palantir-demos-how-ai-can-used-military/",
    "https://www.artificialintelligence-news.com/2023/04/26/ftc-chairwoman-no-ai-exemption-to-existing-laws/",
    "https://www.artificialintelligence-news.com/2023/04/24/bill-gates-ai-teaching-kids-literacy-within-18-months/",
    "https://www.artificialintelligence-news.com/2023/04/21/google-creates-new-ai-division-to-challenge-openai/"
]

session = requests.Session()
pages_content = [] # where we save the scraped articles

for url in article_urls:
    try:
        time.sleep(2) # sleep two seconds for gentle scraping
        response = session.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            article = Article(url)
            article.download() # download HTML of webpage
            article.parse() # parse HTML to extract the article text
            pages_content.append({ "url": url, "text": article.text })
        else:
            print(f"Failed to fetch article at {url}")
    except Exception as e:
        print(f"Error occurred while fetching article at {url}: {e}")

#If an error occurs while fetching an article, we catch the exception and print
#an error message. This ensures that even if one article fails to download,
#the rest of the articles can still be processed.

In [2]:
# We'll use an embedding model to compute our documents' embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

# We'll store the documents and their embeddings in the deep lake vector db
from langchain.vectorstores import DeepLake

# Setup deep lake
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "mbilalshahid"
my_activeloop_dataset_name = "langchain_course_analysis_outline"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


In [3]:
# We split the article texts into small chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts = []
for d in pages_content:
    chunks = text_splitter.split_text(d["text"])
    for chunk in chunks:
        all_texts.append(chunk)

In [4]:
# we add all the chunks to the Deep lake
db.add_texts(all_texts)

|

Dataset(path='hub://mbilalshahid/langchain_course_analysis_outline', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (94, 1536)  float32   None   
    id        text      (94, 1)      str     None   
 metadata     json      (94, 1)      str     None   
   text       text      (94, 1)      str     None   


['2ba1d4ca-5720-11ee-b60b-d83bbf21f498',
 '2ba1d4cb-5720-11ee-98a7-d83bbf21f498',
 '2ba1d4cc-5720-11ee-b082-d83bbf21f498',
 '2ba1d4cd-5720-11ee-b8f9-d83bbf21f498',
 '2ba1d4ce-5720-11ee-9aea-d83bbf21f498',
 '2ba1d4cf-5720-11ee-8297-d83bbf21f498',
 '2ba1d4d0-5720-11ee-9092-d83bbf21f498',
 '2ba1d4d1-5720-11ee-bdca-d83bbf21f498',
 '2ba1d4d2-5720-11ee-a0b8-d83bbf21f498',
 '2ba1d4d3-5720-11ee-9f19-d83bbf21f498',
 '2ba1d4d4-5720-11ee-99ea-d83bbf21f498',
 '2ba1d4d5-5720-11ee-bac4-d83bbf21f498',
 '2ba1d4d6-5720-11ee-b229-d83bbf21f498',
 '2ba1d4d7-5720-11ee-b51c-d83bbf21f498',
 '2ba1d4d8-5720-11ee-9369-d83bbf21f498',
 '2ba1d4d9-5720-11ee-bdf0-d83bbf21f498',
 '2ba1d4da-5720-11ee-9e80-d83bbf21f498',
 '2ba1d4db-5720-11ee-96ea-d83bbf21f498',
 '2ba1d4dc-5720-11ee-a556-d83bbf21f498',
 '2ba1d4dd-5720-11ee-b910-d83bbf21f498',
 '2ba1d4de-5720-11ee-ba1e-d83bbf21f498',
 '2ba1d4df-5720-11ee-8dfa-d83bbf21f498',
 '2ba1d4e0-5720-11ee-a04c-d83bbf21f498',
 '2ba1d4e1-5720-11ee-88e9-d83bbf21f498',
 '2ba1d4e2-5720-

In [5]:
# Get the retriever object from the deep lake db object and set the number
# of retrieved documents to 3
retriever = db.as_retriever()
retriever.search_kwargs['k'] = 3

# We define some variables that will be used inside our custom tool
CUSTOM_TOOL_DOCS_SEPARATOR ="\n---------------\n" # how to join together the retrieved docs to form a single string

# This is the function that defines our custom tool that retrieves relevant
# docs from Deep Lake
def retrieve_n_docs_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)
    texts = [doc.page_content for doc in docs]
    texts_merged = "---------------\n" + CUSTOM_TOOL_DOCS_SEPARATOR.join(texts) + "\n---------------"
    return texts_merged

In [6]:
from langchain.agents.tools import Tool

# We create the tool that uses the "retrieve_n_docs_tool" function
tools = [
    Tool(
        name="Search Private Docs",
        func=retrieve_n_docs_tool,
        description="useful for when you need to answer questions about current events about Artificial Intelligence"
    )
]

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

# let's create the Plan and Execute agent
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [ ]:
# we test the agent
response = agent.run("Write an overview of Artificial Intelligence regulations by governments of country")

In [ ]:
#  Entering new PlanAndExecute chain...

# steps=[Step(value='Research the current state of Artificial Intelligence (AI) regulations in various countries.'), Step(value='Identify the key countries with significant AI regulations or ongoing discussions about AI regulations.'), Step(value='Summarize the AI regulations or discussions in each identified country.'), Step(value='Organize the information by country, providing an overview of the AI regulations in each.'), Step(value='Given the above steps taken, provide an overview of Artificial Intelligence regulations by governments by country.\n')]

In [ ]:
# > Entering new AgentExecutor chain...Action:
# ```
# {
#   "action": "Search Private Docs",
#   "action_input": "current state of Artificial Intelligence regulations in various countries"
# }
# ```
# Observation: ---------------
# “US-based AI developers will likely steal a march on their European competitors given the news that the EU parliamentary committees have green-lit its groundbreaking AI Act, where AI systems will need to be categorized according to their potential for harm from the outset. The US tech approach is typically to experiment first and, once market and product fit is established, to retrofit to other markets and their regulatory framework. This approach fosters innovation whereas EU-based AI developers will need to take note of the new rules and develop systems and processes which may take the edge off their ability to innovate. The UK is adopting a similar approach to the US, although the proximity of the EU market means that UK-based developers are more likely to fall into step with the EU ruleset from the outset. However, the potential to experiment in a safe space – a regulatory sandbox – may prove very attractive.”
# ---------------
# To boost AI innovation, MEPs added exemptions to these rules for research activities and AI components provided under open-source licenses. The new law also promotes regulatory sandboxes – or controlled environments established by public authorities – to test AI before its deployment.

# MEPs want to boost citizens’ right to file complaints about AI systems and receive explanations of decisions based on high-risk AI systems that significantly impact their rights. MEPs also reformed the role of the EU AI Office, which would be tasked with monitoring how the AI rulebook is implemented.

# Tim Wright, Tech and AI Regulatory Partner at London-based law firm Fladgate commented:
# ---------------
# Ryan is a senior editor at TechForge Media with over a decade of experience covering the latest technology and interviewing leading industry figures. He can often be sighted at tech conferences with a strong coffee in one hand and a laptop in the other. If it's geeky, he’s probably into it. Find him on Twitter (@Gadget_Ry) or Mastodon (@gadgetry@techhub.social)

# The Internal Market Committee and the Civil Liberties Committee of the European Parliament have endorsed new transparency and risk-management rules for artificial intelligence systems known as the AI Act.

# This marks a major step in the development of AI regulation in Europe, as these are the first-ever rules for AI. The rules aim to ensure that AI systems are safe, transparent, traceable, and non-discriminatory.

# After the vote, co-rapporteur Brando Benifei (S&D, Italy) said:
# ---------------
# Thought: I have information about the current state of AI regulations in the EU, US, and UK. I can provide a summary of the AI regulations in these regions.
# Action:
# ```
# {
#   "action": "Final Answer",
#   "action_input": "The current state of AI regulations varies across countries. In the European Union, the AI Act has been endorsed by the Internal Market Committee and the Civil Liberties Committee, marking a major step in AI regulation. The AI Act aims to ensure that AI systems are safe, transparent, traceable, and non-discriminatory. In the United States, AI developers typically experiment first and then retrofit their products to other markets and regulatory frameworks, fostering innovation. The UK is adopting a similar approach to the US, but its proximity to the EU market means that UK-based developers are more likely to align with the EU ruleset from the outset. Regulatory sandboxes, or controlled environments established by public authorities, are being promoted to test AI before deployment."
# }
# ```> Finished chain.

In [ ]:
print(response)

In [ ]:
# European Union: The AI Act has been endorsed by the Internal Market Committee and the Civil Liberties Committee, aiming to ensure AI systems are safe, transparent, traceable, and non-discriminatory.

# United States: AI developers typically experiment first and then retrofit their products to other markets and regulatory frameworks, fostering innovation.

# United Kingdom: The UK is adopting a similar approach to the US, but its proximity to the EU market means that UK-based developers are more likely to align with the EU ruleset from the outset. Regulatory sandboxes are being promoted to test AI before deployment.
